# 학생 이동 및 교환 비용 행렬 계산

In [24]:
import pandas as pd
df = pd.read_excel("최종_그룹_배정_결과.xlsx", sheet_name="신규반번호순")
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_우울_T점수,정서적 민감성_우울_수준,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,45,보통,48,보통,32,낮음,62,높음,1,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,2
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,3
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,4
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,5


In [25]:
# 초기 그룹 컬럼 추가하기 (랜덤 숫자 기반)
import numpy as np
df['초기그룹'] = np.random.randint(0, 11, size=len(df))
df['상담필요'] = np.random.randint(0, 2, size=len(df))
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호,초기그룹,상담필요
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,48,보통,32,낮음,62,높음,1,1,3,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,2,2,1
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,3,4,0
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,4,7,0
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,5,2,1


In [26]:
def array_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    이산형 변수의 ideal 대비 actual 차이(diff)와 방향(sign)을 계산하여
    3차원 numpy 배열로 반환하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    selected_discrete_variable : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    diff_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      diff_matrix[g, v, c] = 그룹 g, 변수 v, 카테고리 c 의 diff 값
      diff = ideal_count - actual_count

    sign_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      sign_matrix[g, v, c] = diff의 부호 (+1, 0, -1)

    Notes
    -----
    해당 함수는 기존 딕셔너리 구조를 행렬 구조로 바꿔 compute_discrete_cost 계산을 벡터화하기 위해 수정함
    모든 변수중 가장 많은 카테고리 수에 맞춰 0으로 패딩하여 shape이 동일하도록 구성함
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np
    # 확인해보니 반환값이 그룹 번호 순으로 안나와서 정렬 추가
    groups = sorted(list(group_freq.keys()))
    n_groups = len(groups)
    vars_list = selected_discrete_variable
    n_vars = len(vars_list)
    var_to_idx = {var: idx for idx, var in enumerate(vars_list)} # 예시 {성별: 0, 상담필요: 1}
    category_to_idx = {}
    max_categories = 0
    for var in vars_list:
        categories = set(ideal_freq['population'][var].keys()) # 예시 var 성별이면 {1,2}, 상담필요면 {0,1}
        categories = sorted(list(categories))
        category_to_idx[var] = {cat: idx for idx, cat in enumerate(categories)} # 예시 {성별: {1:0, 2:1}, 상담필요: {0:0, 1:1}}
        if len(categories) > max_categories:
            max_categories = len(categories) # 가장 많은 카테고리 수 갱신
    # 행렬 초기화
    diff_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    sign_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    # 행렬 채우기
    try:
        for g_idx, g in enumerate(groups):
            for var in vars_list:
                v_idx = var_to_idx[var]
                cats = category_to_idx[var]
                for cat, c_idx in cats.items():
                    ideal_count = ideal_freq['population'][var].get(cat, 0)
                    acutual_count = group_freq[g][var].get(cat, 0)
                    diff = ideal_count - acutual_count
                    diff_matrix[g_idx, v_idx, c_idx] = diff
                    sign_matrix[g_idx, v_idx, c_idx] = np.sign(diff)
    except Exception as e:
        print("Error during array_diff_and_sign execution:", str(e))
        raise e
    return diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups

In [27]:
# ideal_freq, group_freq, selected_discrete_variable
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
ideal_freq = compute_ideal_discrete_freq(df, ['성별', '상담필요'])
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
print("Ideal Frequency:", ideal_freq)
print("Group Frequency:", group_freq)

Ideal Frequency: {'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.727272727272727, 1: 12.909090909090908}}}
Group Frequency: {3: {'성별': {1: 9, 2: 9}, '상담필요': {1: 10, 0: 8}}, 2: {'성별': {2: 16, 1: 14}, '상담필요': {0: 16, 1: 14}}, 4: {'성별': {2: 16, 1: 14}, '상담필요': {1: 18, 0: 12}}, 7: {'성별': {2: 12, 1: 11}, '상담필요': {1: 13, 0: 10}}, 1: {'성별': {2: 13, 1: 11}, '상담필요': {0: 14, 1: 10}}, 9: {'성별': {2: 12, 1: 10}, '상담필요': {1: 17, 0: 5}}, 10: {'성별': {1: 18, 2: 15}, '상담필요': {0: 17, 1: 16}}, 6: {'성별': {2: 18, 1: 13}, '상담필요': {0: 20, 1: 11}}, 8: {'성별': {2: 14, 1: 11}, '상담필요': {0: 16, 1: 9}}, 0: {'성별': {1: 23, 2: 13}, '상담필요': {0: 21, 1: 15}}, 5: {'성별': {1: 13, 2: 8}, '상담필요': {0: 12, 1: 9}}}


In [28]:
diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups = array_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
print(diff_matrix) # 그룹별 변수별 카테고리별 diff 값
print(sign_matrix) # 그룹별 변수별 카테고리별 sign 값
print(var_to_idx) # 변수명과 인덱스 매핑
print(category_to_idx) # 변수별 카테고리명과 인덱스 매핑
print(groups) # 그룹 번호 리스트

[[[-9.63636364  0.27272727]
  [-7.27272727 -2.09090909]]

 [[ 2.36363636  0.27272727]
  [-0.27272727  2.90909091]]

 [[-0.63636364 -2.72727273]
  [-2.27272727 -1.09090909]]

 [[ 4.36363636  4.27272727]
  [ 5.72727273  2.90909091]]

 [[-0.63636364 -2.72727273]
  [ 1.72727273 -5.09090909]]

 [[ 0.36363636  5.27272727]
  [ 1.72727273  3.90909091]]

 [[ 0.36363636 -4.72727273]
  [-6.27272727  1.90909091]]

 [[ 2.36363636  1.27272727]
  [ 3.72727273 -0.09090909]]

 [[ 2.36363636 -0.72727273]
  [-2.27272727  3.90909091]]

 [[ 3.36363636  1.27272727]
  [ 8.72727273 -4.09090909]]

 [[-4.63636364 -1.72727273]
  [-3.27272727 -3.09090909]]]
[[[-1.  1.]
  [-1. -1.]]

 [[ 1.  1.]
  [-1.  1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1. -1.]
  [ 1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1. -1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1. -1.]]

 [[-1. -1.]
  [-1. -1.]]]
{'성별': 0, '상담필요': 1}
{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}
[0, 1, 2, 3,

In [29]:
# 기존 딕셔너리로 계산하는 함수 가지고 와서 array_diff_and_sign와 동일한지 확인
import pprint
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
pprint.pprint(group_freq)
print(ideal_freq)
def compute_group_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    각 그룹별 이산형 변수의 차이(diff)와 방향(sign)을 계산하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    discrete_vars : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    group_diff_cost : dict
        각 그룹의 변수별 diff와 sign을 담은 딕셔너리
        구조 예시:
        {
            0: {
                '성별_명렬표_diff': {1: 7.5, 2: 9.5},
                '성별_명렬표_sign': {1: +1.0, 2: +1.0},
                '상담 필요_diff': {0: 15.0, 1: 2.0},
                '상담 필요_sign': {0: +1.0, 1: +1.0}
            },
            1: { ... },
            ...
        }

    Notes
    -----
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np

    group_diff_cost = {}
    try:
        for g in group_freq.keys():
            group_diff_cost[g] = {}

            for var in selected_discrete_variable:
                # diff, sign 각각을 별도 dict로 저장
                diff_dict = {}
                sign_dict = {}

                # 각 범주(key)별로 ideal 대비 actual 차이 계산
                for key, ideal_count in ideal_freq['population'][var].items():
                    actual_count = group_freq[g][var].get(key, 0)
                    diff = ideal_count - actual_count
                    diff_dict[key] = diff
                    sign_dict[key] = np.sign(diff) # 이상치보다 작으면 +1, 크면 -1, 같으면 0

                # 변수별 결과를 group_diff_cost에 저장
                group_diff_cost[g][f'{var}_diff'] = diff_dict
                group_diff_cost[g][f'{var}_sign'] = sign_dict
    except Exception as e:
        print("Error during compute_group_diff_and_sign execution:", str(e))
        raise e
    return group_diff_cost
group_diff_cost = compute_group_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
import pprint
pprint.pprint(group_diff_cost)
# 출력값 확인해보니 diff_matrix와 group_diff_cost와 값은 동일하나 그룹 순서가 안맞아서 array_diff_and_sign에 그룹 정렬 추가함
# 행렬의 변수 순서가 다르긴 하지만 이건 var_to_idx로 매핑 가능함

{0: {'상담필요': {0: 21, 1: 15}, '성별': {1: 23, 2: 13}},
 1: {'상담필요': {0: 14, 1: 10}, '성별': {1: 11, 2: 13}},
 2: {'상담필요': {0: 16, 1: 14}, '성별': {1: 14, 2: 16}},
 3: {'상담필요': {0: 8, 1: 10}, '성별': {1: 9, 2: 9}},
 4: {'상담필요': {0: 12, 1: 18}, '성별': {1: 14, 2: 16}},
 5: {'상담필요': {0: 12, 1: 9}, '성별': {1: 13, 2: 8}},
 6: {'상담필요': {0: 20, 1: 11}, '성별': {1: 13, 2: 18}},
 7: {'상담필요': {0: 10, 1: 13}, '성별': {1: 11, 2: 12}},
 8: {'상담필요': {0: 16, 1: 9}, '성별': {1: 11, 2: 14}},
 9: {'상담필요': {0: 5, 1: 17}, '성별': {1: 10, 2: 12}},
 10: {'상담필요': {0: 17, 1: 16}, '성별': {1: 18, 2: 15}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.727272727272727, 1: 12.909090909090908}}}
{0: {'상담필요_diff': {0: -7.272727272727273, 1: -2.0909090909090917},
     '상담필요_sign': {0: -1.0, 1: -1.0},
     '성별_diff': {1: -9.636363636363637, 2: 0.2727272727272734},
     '성별_sign': {1: -1.0, 2: 1.0}},
 1: {'상담필요_diff': {0: -0.2727272727272734, 1: 2.9090909090909083},
     '상담필요_sign': {0: -1.0, 1: 1.0}

In [30]:
# 출발 그룹 설정이 제대로 되는지 확인
from cost_group_move_v3 import compute_group_total_cost
pprint.pprint(group_freq)
print(ideal_freq)
group_total_cost, _ = compute_group_total_cost(ideal_freq, group_freq, ['성별', '상담필요'])
pprint.pprint(group_total_cost)
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

{0: {'상담필요': {0: 21, 1: 15}, '성별': {1: 23, 2: 13}},
 1: {'상담필요': {0: 14, 1: 10}, '성별': {1: 11, 2: 13}},
 2: {'상담필요': {0: 16, 1: 14}, '성별': {1: 14, 2: 16}},
 3: {'상담필요': {0: 8, 1: 10}, '성별': {1: 9, 2: 9}},
 4: {'상담필요': {0: 12, 1: 18}, '성별': {1: 14, 2: 16}},
 5: {'상담필요': {0: 12, 1: 9}, '성별': {1: 13, 2: 8}},
 6: {'상담필요': {0: 20, 1: 11}, '성별': {1: 13, 2: 18}},
 7: {'상담필요': {0: 10, 1: 13}, '성별': {1: 11, 2: 12}},
 8: {'상담필요': {0: 16, 1: 9}, '성별': {1: 11, 2: 14}},
 9: {'상담필요': {0: 5, 1: 17}, '성별': {1: 10, 2: 12}},
 10: {'상담필요': {0: 17, 1: 16}, '성별': {1: 18, 2: 15}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.727272727272727, 1: 12.909090909090908}}}
{0: -150.0495867768595,
 1: 14.049586776859497,
 2: -14.198347107438018,
 3: 78.56198347107437,
 4: -30.77685950413224,
 5: 46.19834710743801,
 6: -57.91735537190083,
 7: 21.090909090909083,
 8: 15.173553719008254,
 9: 72.36363636363635,
 10: -44.74380165289257}
Source Group Index with Minimum Total Cost: 

In [31]:
# 출발 학생 목록 가지고오기
source_group_df = df[df['초기그룹']==source_group_idx]
print(source_group_df['상담필요'].value_counts())
print(source_group_df['성별'].value_counts())

상담필요
0    21
1    15
Name: count, dtype: int64
성별
1    23
2    13
Name: count, dtype: int64


In [32]:
# 타깃그룹 설정
groups = df['초기그룹'].unique()
target_group_list = [g for g in groups if g != source_group_idx]
print("Target Group List (excluding source group):", target_group_list)

Target Group List (excluding source group): [3, 2, 4, 7, 1, 9, 10, 6, 8, 5]


In [33]:
var_to_idx

{'성별': 0, '상담필요': 1}

In [34]:
category_to_idx

{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}

In [35]:
diff_matrix
# 해당 배열은 아래와 같음
# G, V, C 순서로 되어있음
# G : 그룹 번호
# V : 변수 번호 (0: 성별, 1: 상담필요)
# C : 카테고리 번호 (성별: 0-남,1-여 / 상담필요: 0-필요없음,1-필요함)

array([[[-9.63636364,  0.27272727],
        [-7.27272727, -2.09090909]],

       [[ 2.36363636,  0.27272727],
        [-0.27272727,  2.90909091]],

       [[-0.63636364, -2.72727273],
        [-2.27272727, -1.09090909]],

       [[ 4.36363636,  4.27272727],
        [ 5.72727273,  2.90909091]],

       [[-0.63636364, -2.72727273],
        [ 1.72727273, -5.09090909]],

       [[ 0.36363636,  5.27272727],
        [ 1.72727273,  3.90909091]],

       [[ 0.36363636, -4.72727273],
        [-6.27272727,  1.90909091]],

       [[ 2.36363636,  1.27272727],
        [ 3.72727273, -0.09090909]],

       [[ 2.36363636, -0.72727273],
        [-2.27272727,  3.90909091]],

       [[ 3.36363636,  1.27272727],
        [ 8.72727273, -4.09090909]],

       [[-4.63636364, -1.72727273],
        [-3.27272727, -3.09090909]]])

In [36]:
# 배열의 차원 예시는 아래와 같다
# G : 그룹 번호
diff_matrix[source_group_idx] # 출발 그룹의 변수별 카테고리별 diff 값

array([[-9.63636364,  0.27272727],
       [-7.27272727, -2.09090909]])

In [37]:
# V : 변수 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값
s_row = source_group_df.iloc[0]
print(s_row[['성별', '상담필요']])
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    print(f"Variable: {var}, Index: {v_idx}")
    print(diff_matrix[source_group_idx, v_idx]) # 출발 그룹의 첫번째 변수(성별)의 카테고리별 diff 값

성별      2
상담필요    1
Name: 18, dtype: object
Variable: 성별, Index: 0
[-9.63636364  0.27272727]
Variable: 상담필요, Index: 1
[-7.27272727 -2.09090909]


In [38]:
# C : 카테고리 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값 중
# 해당 학생이 속한 카테고리의 diff 값
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    cat_value = s_row[var]
    c_idx = category_to_idx[var][cat_value]
    print(f"Variable: {var}, Category Value: {cat_value}, Category Index: {c_idx}")
    print(diff_matrix[source_group_idx, v_idx, c_idx]) # 출발 그룹의 첫번째 변수(성별)의 해당 카테고리의 diff 값

Variable: 성별, Category Value: 2, Category Index: 1
0.2727272727272734
Variable: 상담필요, Category Value: 1, Category Index: 1
-2.0909090909090917


In [39]:
# 이동 전 모든 그룹의 diff_matrix 확인
for group_idx in range(diff_matrix.shape[0]):
    print(f"Group Index: {group_idx}")
    print(diff_matrix[group_idx])

Group Index: 0
[[-9.63636364  0.27272727]
 [-7.27272727 -2.09090909]]
Group Index: 1
[[ 2.36363636  0.27272727]
 [-0.27272727  2.90909091]]
Group Index: 2
[[-0.63636364 -2.72727273]
 [-2.27272727 -1.09090909]]
Group Index: 3
[[4.36363636 4.27272727]
 [5.72727273 2.90909091]]
Group Index: 4
[[-0.63636364 -2.72727273]
 [ 1.72727273 -5.09090909]]
Group Index: 5
[[0.36363636 5.27272727]
 [1.72727273 3.90909091]]
Group Index: 6
[[ 0.36363636 -4.72727273]
 [-6.27272727  1.90909091]]
Group Index: 7
[[ 2.36363636  1.27272727]
 [ 3.72727273 -0.09090909]]
Group Index: 8
[[ 2.36363636 -0.72727273]
 [-2.27272727  3.90909091]]
Group Index: 9
[[ 3.36363636  1.27272727]
 [ 8.72727273 -4.09090909]]
Group Index: 10
[[-4.63636364 -1.72727273]
 [-3.27272727 -3.09090909]]


In [40]:
# 출발 그룹에서 모든 학생들이 도착 그룹으로 이동했을때 diff_matrix 계산
'''
어떻게 생각해야할까?
출발 그룹 중 하나의 학생이
도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?
그면 차원이 어떻게 될까?
    출발 그룹 학생 수 만큼 차원이 늘어나고
    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?
    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?
    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데
'''
# 우선 차원 틀 먼저 만들어보자
# N : 출발 그룹 학생 수
# G : 도착 그룹 수 (출발 그룹 제외해야하지만 계산을 위해 모든 그룹 포함)
# V : 변수 수
# C : 가장 많은 카테고리 수

'\n어떻게 생각해야할까?\n출발 그룹 중 하나의 학생이\n도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?\n그면 차원이 어떻게 될까?\n    출발 그룹 학생 수 만큼 차원이 늘어나고\n    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?\n    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?\n    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데\n'

In [68]:
# 이동 후 diff_matrix 배열 반영
N = source_group_df.shape[0]
G = len(target_group_list)
V = diff_matrix.shape[1]
C = diff_matrix.shape[2]
print(f"N (출발 그룹 학생 수): {N}, G (전체 그룹 수): {G}, V (변수 수): {V}, C (최대 카테고리 수): {C}")

N (출발 그룹 학생 수): 36, G (전체 그룹 수): 10, V (변수 수): 2, C (최대 카테고리 수): 2


In [73]:
# 출발 그룹 df 인덱스와 행렬 인덱스 매핑
studentidx_to_arrayidx = {}
for idx, student_idx in enumerate(source_group_df.index):
    studentidx_to_arrayidx[student_idx] = idx
print("Source Group DataFrame Index to Array Index Mapping:", studentidx_to_arrayidx)
arrayidx_to_studentidx = {}
for student_idx, array_idx in studentidx_to_arrayidx.items():
    arrayidx_to_studentidx[array_idx] = student_idx
print("Array Index to Source Group DataFrame Index Mapping:", arrayidx_to_studentidx)

Source Group DataFrame Index to Array Index Mapping: {18: 0, 20: 1, 23: 2, 37: 3, 42: 4, 49: 5, 50: 6, 52: 7, 73: 8, 108: 9, 111: 10, 113: 11, 116: 12, 123: 13, 144: 14, 151: 15, 158: 16, 163: 17, 166: 18, 169: 19, 172: 20, 173: 21, 179: 22, 190: 23, 198: 24, 214: 25, 221: 26, 232: 27, 239: 28, 248: 29, 254: 30, 255: 31, 264: 32, 271: 33, 276: 34, 277: 35}
Array Index to Source Group DataFrame Index Mapping: {0: 18, 1: 20, 2: 23, 3: 37, 4: 42, 5: 49, 6: 50, 7: 52, 8: 73, 9: 108, 10: 111, 11: 113, 12: 116, 13: 123, 14: 144, 15: 151, 16: 158, 17: 163, 18: 166, 19: 169, 20: 172, 21: 173, 22: 179, 23: 190, 24: 198, 25: 214, 26: 221, 27: 232, 28: 239, 29: 248, 30: 254, 31: 255, 32: 264, 33: 271, 34: 276, 35: 277}


In [71]:
# G가 결국 어떠한 값이 정확한 도착 그룹 번호인지 명확하지 않아서
# 맵칭 데이터가 필요함
targetgroup_to_arrayidx = {}
for idx, g in enumerate(target_group_list):
    targetgroup_to_arrayidx[g] = idx
print(targetgroup_to_arrayidx)
# key값과 value값이 바뀐것도 필요한가?
arrayidx_to_targetgroup = {}
for arrayidx, target_group in zip(range(G), target_group_list):
    arrayidx_to_targetgroup[arrayidx] = target_group
print(arrayidx_to_targetgroup)

{3: 0, 2: 1, 4: 2, 7: 3, 1: 4, 9: 5, 10: 6, 6: 7, 8: 8, 5: 9}
{0: 3, 1: 2, 2: 4, 3: 7, 4: 1, 5: 9, 6: 10, 7: 6, 8: 8, 9: 5}


In [74]:
affter_delta_matrix = np.zeros((N, G, V, C), dtype=float)  # N x G x V x C 형태의 배열 초기화

for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    stident_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(G):
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            affter_delta_matrix[stident_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = -1.0

In [82]:
affter_delta_matrix[:, 0, :, :]

array([[[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1., 

In [79]:
for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    print(s_row['성별'])

2
1
1
1
1
1
1
2
2
2
1
1
1
1
1
1
1
2
1
1
1
2
2
2
2
1
1
2
1
2
2
2
1
1
1
1
